In [36]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'uniform_dense_join_small_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/books_join_small_1/outputs/results/run0', '/home/chesetti/Repos/learned_merge_cleanup/sponge/books_join_small_1/outputs/results/run1', '/home/chesetti/Repos/learned_merge_cleanup/sponge/books_join_small_1/outputs/results/run2']


In [37]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_index_build_duration_ns', 'result.inner_index_size',
       'result.outer_disk_fetch', 'result.outer_index_build_duration',
       'result.outer_index_size', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.type', 'spec.inner_table',
       'spec.key_size', 'spec.key_type', 'spec.load_sstable_in_mem',
       'spec.name', 'spec.num_threads', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk',
       'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [38]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name   btree1024  btree2048  btree256  btree512  hash_join  \
spec.common_key                                                        
10                37105567   32730542  35826760  40161801  100819522   
20                16586303   28320466  36978270  17745191   98259199   
30                14488076   21124485  23906286  21390075   97700430   
40                12752240    9465462  20886244  12010865   90689329   
50                14685209    8179007  10507388  13256389  104202647   
60                 7135944   17042873  18855304  14341385   80095434   
70                12280322    6130231   8519273  12104941   72683573   
80                11603005   11583147  15499578  12824073   69899493   
90                 6148973   35145956  14504118  12467808   65426349   
100                7569759    5377094   7501290  30644811   76767513   

spec.algo_name    pgm1024    pgm128    pgm256    pgm512     pgm64        sj  \
spec.common_key                                         

In [39]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

     indexes  memory_bytes
0  btree1024       20592.0
1  btree2048        4120.0
2   btree256       70032.0
3   btree512       37072.0
4    pgm1024         264.0
5     pgm128         728.0
6     pgm256         456.0
7     pgm512         328.0
8      pgm64        1696.0


In [40]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))

In [41]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
print(inner_index_fetch)

spec.algo_name   btree1024  btree2048  btree256  btree512  hash_join  pgm1024  \
spec.common_key                                                                 
10                     976        244      3904      1952       3904     3900   
20                     976        244      3904      1952       3904     3899   
30                     976        244      3903      1952       3904     3897   
40                     976        244      3898      1952       3904     3890   
50                     976        244      3871      1952       3904     3873   
60                     976        244      3852      1951       3904     3833   
70                     976        244      3779      1951       3904     3792   
80                     976        244      3728      1948       3904     3739   
90                     976        244      3671      1947       3904     3679   
100                    976        244      3589      1943       3904     3573   

spec.algo_name   pgm128  pg

In [42]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 30])
print(result_checksum.loc[result_checksum['spec.common_key'] == 30]['result.checksum'].unique())

common_key: 10 checksum: ['9A0745C81F145039CC08E6DB20692E2F'] OK
common_key: 20 checksum: ['56A3B5D3D032A2AFD0741AB3EFA352E7'] OK
common_key: 30 checksum: ['0893DFC694E5CD10E330D39C93759503'] OK
common_key: 40 checksum: ['14D73137C48284B02B60E9E08A43CB2F'] OK
common_key: 50 checksum: ['A8A4696F23CC2BAC931535A5158A2B4A'] OK
common_key: 60 checksum: ['03F103D5B3908B242D7CBA89DA092627'] OK
common_key: 70 checksum: ['F88FC70D71930F1B047A3CD00951DB70'] OK
common_key: 80 checksum: ['C1A4FFD2B3EEBDF69D43F96070106B60'] OK
common_key: 90 checksum: ['27B292B921D1544D3291C5467954C940'] OK
common_key: 100 checksum: ['36CD418404FA814E2B289C7C68A31C54'] OK
     spec.common_key spec.algo_name                   result.checksum
105               30      btree1024  0893DFC694E5CD10E330D39C93759503
225               30      btree1024  0893DFC694E5CD10E330D39C93759503
345               30      btree1024  0893DFC694E5CD10E330D39C93759503
111               30      btree2048  0893DFC694E5CD10E330D39C93759503